In [ ]:
# THIS CELL WAS AUTO-GENERATED BY PYFLYBY


import numpy as np
import pandas as pd

# END AUTO-GENERATED BLOCK

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level=logging.INFO)
import tsdm

In [ ]:
ds = tsdm.datasets.KiwiBenchmark()

In [ ]:
ds.metadata.index

In [ ]:
from enum import Enum
from typing import NamedTuple, TypedDict


class Weekday(Enum):
    MONDAY: int = 1
    TUESDAY: int = 2
    WEDNESDAY: int = 3
    THURSDAY: int = 4
    FRIDAY: int = 5
    SATURDAY: int = 6
    SUNDAY: int = 7


dict(Weekday)

In [ ]:
ds.metadata.iloc[-1]

In [ ]:
df[::-1]

In [ ]:
next(iter(reversed(pd.Series([1, 2, 3, 4, 5]))))

In [ ]:
from dataclasses import dataclass


@dataclass
class Foo:
    name: str

    def __lt__(self, other):
        print(f"{self} < {other} called on {self}")

    def __gt__(self, other):
        print(f"{self} > {other} called on {self}")


x = Foo("x")
y = Foo("y")
z = Foo("z")
print(x, y, z)

1 < x
x < 1
x < y
y < x
x < y < z

In [ ]:
import types

In [ ]:
x.value

In [ ]:
x = pd.Series(index=ds.metadata.index, dtype=bool)
x[:] = np.random.randn(*x.shape) > 0.5
ds.timeseries.loc[x]

In [ ]:
ds = tsdm.datasets.DampedPendulum_Ansari2023()

In [ ]:
from tsdm.tasks.utils import FixedSliceSampleGenerator

sampler = FixedSliceSampleGenerator(ds.table, input_slice=slice(None, 10.0))

In [ ]:
sample = next(iter(sampler))

In [ ]:
import pickle

In [ ]:
with open("foo.pickle", "wb") as file:
    pickle.dump(sample, file)

In [ ]:
with open("foo.pickle", "rb") as file:
    rec = pickle.load(file)

In [ ]:
from tsdm.types.protocols import NTuple

In [ ]:
from typing import NamedTuple


class Point(NamedTuple):
    x: float
    y: float


issubclass(Point, NTuple)

In [ ]:
ds.table.index

In [ ]:
print(tsdm.utils.strings.repr_dataclass(sampelr))

In [ ]:
sampelr.__repr__

In [ ]:
from dataclasses import dataclass
from typing import NamedTuple, final

from pandas import DataFrame
from sklearn.model_selection import train_test_split
from torch import Tensor
from torch.utils.data import Dataset as TorchDataset

from tsdm import datasets
from tsdm.random.samplers import RandomSampler, Sampler
from tsdm.tasks.base import SplitID, TimeSeriesTask
from tsdm.utils.data import folds_as_frame, is_partition
from tsdm.utils.data.datasets import KW_ONLY, DataFrame2Dataset
from tsdm.utils.strings import repr_namedtuple


class Inputs(NamedTuple):
    r"""A single sample of the data."""

    t: Tensor
    x: Tensor
    t_target: Tensor

    def __repr__(self) -> str:
        r"""Return string representation."""
        return repr_namedtuple(self)

In [ ]:
ds = tsdm.datasets.MIMIC_IV(reset=True, version=2.0)

In [ ]:
ds = tsdm.datasets.KiwiBenchmark()
ts = ds.timeseries

In [ ]:
ts.index.get_level_values(levels=slice(None, -1))

In [ ]:
ts.reset_index(level=-1).index.unique()

In [ ]:
hasattr(type(ds), "timeseries")

In [ ]:
from getpass import getpass

import tsdm
from tsdm.utils.remote import download

In [ ]:
ds = tsdm.datasets.MIMIC_III(initialize=False, version="1.2")

In [ ]:
ds.download()

In [ ]:
ds.HOME_URL

In [ ]:
tuple(map(int, "1.3".split("."))) <= (1, 4)

In [ ]:
from typing import Literal

In [ ]:
x = Literal["a", "b"]

In [ ]:
from typing import get_args

In [ ]:
get_args(x)

In [ ]:
url = ds.BASE_URL.format(version="1.4")
fname = "mimic-iii-clinical-database-1.4.zip"

In [ ]:
username = input("MIMIC-III username: ")
password = getpass(prompt="MIMIC-III password: ", stream=None)

In [ ]:
headers = {
    "User-Agent": "Wget/1.21.2",
}

In [ ]:
download(url, fname, headers=headers, username=username, password=password)

In [ ]:
self = MIMIC_III_DeBrouwer2019()

In [ ]:
x: [1, 2] = 2

In [ ]:
"0".isidentifier()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(16, 6, figsize=(20, 32), constrained_layout=True, sharey=True)

for col, ax in zip(self.timeseries, axes.flatten()):
    self.timeseries[col].hist(ax=ax, density=True, log=True, bins=20)
    ax.set_ylim(10**-6, 1)

In [ ]:
%matplotlib inline

(self.metadata["min"] == 0).mean()

In [ ]:
import pandas as pd
from pandas import DataFrame

In [ ]:
self.LOGGER.info("Loading main file.")
ts = pd.read_csv(self.rawdata_paths["complete_tensor.csv"], index_col=0)

# Check shape.
if ts.shape != self.rawdata_shapes["complete_tensor.csv"]:
    raise ValueError(
        f"The {ts.shape=} is not correct."
        "Please apply the modified preprocessing using bin_k=2, as outlined in"
        "the appendix. The resulting tensor should have 3082224 rows and 7 columns."
    )

ts = ts.astype(self.rawdata_schemas["complete_tensor.csv"]).sort_values(
    by=["UNIQUE_ID", "TIME_STAMP"]
)

In [ ]:
means = ts.groupby("LABEL_CODE").mean()["VALUENUM"].rename("MEANS")
stdvs = ts.groupby("LABEL_CODE").std()["VALUENUM"].rename("STDVS")
stats = (
    DataFrame([means, stdvs])
    .T.reset_index()
    .astype({
        "LABEL_CODE": "int16",
        "MEANS": "float32",
        "STDVS": "float32",
    })
)

In [ ]:
ts = (
    ts[["UNIQUE_ID", "TIME_STAMP", "LABEL_CODE", "VALUENUM"]]
    .reset_index(drop=True)
    .set_index(["UNIQUE_ID", "TIME_STAMP"])
    .pivot(columns="LABEL_CODE", values="VALUENUM")
    .sort_index()
    .sort_index(axis=1)
)

In [ ]:
ts.describe().T

In [ ]:
ts.mean()